# Sample answer to Activity 3

*Last updated by Christian Cahig on 2025-10-06*

## Imports

In [1]:
import math as mt

import scipy.optimize as spo

## Scenario

You are a power systems engineer tasked to analyze two customer loads and served at a common bus.
The bus is at the receiving end of a feeder line extending from the utility system.

One of the loads is estimated to be 421 kilowatts at a lagging power factor of 0.80,
while the other is rated 317 kilovars at a lagging power factor of 0.77.

Records show that the feeder line impedance is rated 0.95 + j2.47 ohms.
If it is reasonable to assume that the utility system's RMS voltage hovers around 49 kilovolts,
at what RMS voltage are the loads served?

You are to approach this as a root-finding problem,
and so must first derive a function $f\!\left(V\right)$
whose root corresponds to the desired quantity.

## Modelling

The scenario describes a radial topology:
power flows from where the feeder line connects to the utility system,
through the feeder line,
and to the common bus.
See [./fig-01.png](./fig-01.png).

Without any more information,
we assume we are dealing with steady-state sinusoidal signals.
This allows us to use frequency-domain analysis via phasor representation.
Furthermore, we can set the voltage phasor at the common bus to have the reference phase angle,
and the voltage phasor "upstream" we can set to have some phase angle $\theta$.
As such, we can arrive at the following relation:

$$
49.0 \angle \theta = V \angle 0 + (0.95 + j2.47) I_{\text{load}},
$$

where $V$ is the magnitude (in kilovolts) of the voltage phasor at the common bus,
and $I_{\text{load}}$ is the total current (in kiloamperes) through the feeder line,
as induced by the loads.
Moreover,

$$
\left(V \angle 0\right) \operatorname{conj}\!\left( I_{\text{load}} \right)
=
\frac{0.421}{0.8} \angle\arccos\!\left(0.8\right)
+
\frac{0.317}{\sin\!\left(\arccos\!\left(0.77\right)\right)} \angle\arccos\!\left(0.77\right).
$$

It is then straightforward to arrive at

$$
49^{2} = \left(V + \frac{2.326274397}{V}\right)^{2} + \left(\frac{1.383680433}{V}\right)^{2},
$$

and so a plausible univariate function to root-find is

$$
f\!\left(V\right) = 
\left(V + \frac{2.326274397}{V}\right)^{2} + \left(\frac{1.383680433}{V}\right)^{2}
- 49^{2}.
$$

## Secant method

Construct a Python function `f()` that implements $f\!\left(V\right)$.
`f()` takes a single positional argument `x` representing $V$.

In [2]:
def f(x):
    return (x + (2.326274397 / x))**2 + (1.383680433 / x)**2 - 49.**2

Use the secant method to find a root of $f\!\left(V\right)$.
Use an iteration budget of 100 and a tolerance of $10^{-7}$.

Define the following variables.
- `X0`, to store the initial guess
- `MAX_ITERS`, to store the iteration budget
- `X_TOL` to store the tolerance

Run
[`scipy.optimize.newton()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.newton.html)
such that you get the (approximate) root as well as information concerning the run.
Store the root and the information in variables `p_sc` and `p_sc_info`, respectively.
Make sure to verify if the returned root is indeed a root.

In [3]:
X0 = 40.0
MAX_ITERS = 100
X_TOL = 1e-7

p_sc, p_sc_info = spo.newton(
    f, X0, fprime=None,
    tol=X_TOL, maxiter=MAX_ITERS,
    full_output=True, disp=False
)

## Results

Do not modify nor remove the following code cell.

In [4]:
print(f"Finding a root from {X0} kilovolts:")
print(p_sc_info)
print(f"Residual value: {f(p_sc)}")

Finding a root from 40.0 kilovolts:
      converged: True
           flag: converged
 function_calls: 7
     iterations: 6
           root: 48.952470764994494
         method: secant
Residual value: -9.094947017729282e-13


In [5]:
print(f"The RMS voltage at the common bus is {p_sc} kilovolts.")

The RMS voltage at the common bus is 48.952470764994494 kilovolts.


## Some things to consider

- Due to the radial topology,
it is reasonable to expect
the RMS voltage at the common bus to be smaller in magnitude
than that at which the feeder is coupled to the utility system.

- It is therefore practically reasonable to set the initial estimate to something near (or slightly less than)
  the utility system voltage level. 

- Try to make the convergence criterion stricter
and verify if the run indeed takes more iterations.

- What about a different $f\!\left(V\right)$?

- Does $f\!\left(V\right)$ have other roots?